# Подготовка рекомендательной системы курсов для пользователей

## ЗАДАЧИ
Создать рекомендательную систему курсов в онлайн-школе MasterMind,\
благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

## КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
1) Подготовить и проанализировать данные с помощью SQL.
2) Обработать данные средствами Python.
3) Составить итоговую таблицу, которая станет основой рекомендательной системы.

## 1) Работа с базой данных

Подсчитаем количество пользователей, купивших курсы:

WITH courses AS\
( \
SELECT \
    &emsp;c.user_id,\
    &emsp;c.id AS cart_id,\
    &emsp;c.state,\
    &emsp;ci.resource_type,\
    &emsp;ci.resource_id\
FROM\
    &emsp;final.carts c\
    &emsp;JOIN final.cart_items ci ON c.id = ci.cart_id\
)

SELECT\
    &emsp;COUNT(DISTINCT user_id) AS users_cnt\
FROM courses\
WHERE state = 'successful' AND resource_type = 'Course'

### 49006 клиентов покупали курсы

Подсчитаем общее количество различных курсов:

WITH courses AS\
(\
SELECT \
    &emsp;c.user_id,\
    &emsp;c.id AS cart_id,\
    &emsp;c.state,\
    &emsp;ci.resource_type,\
    &emsp;ci.resource_id\
FROM \
    &emsp;final.carts c\
    &emsp;JOIN final.cart_items ci ON c.id = ci.cart_id\
)

SELECT\
    &emsp;COUNT(DISTINCT resource_id)\
FROM courses\
WHERE resource_type = 'Course'

### Всего 127 курсов

Подсчитаем среднее число купленных курсов на одного клиента:

WITH courses AS\
(\
SELECT \
    &emsp;c.user_id,\
    &emsp;c.id AS cart_id,\
    &emsp;c.state,\
    &emsp;ci.resource_type,\
    &emsp;ci.resource_id\
FROM \
    &emsp;final.carts c\
    &emsp;JOIN final.cart_items ci ON c.id = ci.cart_id\
)

SELECT\
   &emsp;AVG(courses_cnt)\
FROM\
(SELECT\
    &emsp;user_id,\
    &emsp;COUNT(resource_id) as courses_cnt\
FROM courses\
WHERE resource_type = 'Course'\
AND state = 'successful'\
GROUP BY user_id) courses_per_user

### Один клиент купил в среднем 1.44 курса

Подсчитаем количество клиентов, купивших больше одного курса:

WITH courses AS\
(\
SELECT \
    &emsp;c.user_id,\
    &emsp;c.id AS cart_id,\
    &emsp;c.state,\
    &emsp;ci.resource_type,\
    &emsp;ci.resource_id\
FROM \
    &emsp;final.carts c \
    &emsp;JOIN final.cart_items ci ON c.id = ci.cart_id\
)

SELECT\
    &emsp;COUNT(DISTINCT user_id) \
FROM\
(SELECT\
    &emsp;user_id,\
    &emsp;COUNT(DISTINCT resource_id) as courses_cnt\
FROM courses\
WHERE resource_type = 'Course' \
AND state = 'successful'\
GROUP BY user_id\
HAVING COUNT(DISTINCT resource_id) > 1) activ_users

### 12656 клиентов купили больше одного курса

Подготовим файл с данными по продажам курсов в разрезе пользователей (купивших более одного курса):

WITH courses AS\
(\
SELECT \
    &emsp;c.user_id,\
    &emsp;c.id AS cart_id,\
    &emsp;c.state,\
    &emsp;ci.resource_type,\
    &emsp;ci.resource_id\
FROM \
    &emsp;final.carts c \
    &emsp;JOIN final.cart_items ci ON c.id = ci.cart_id\
)

SELECT DISTINCT\
    &emsp;user_id,\
    &emsp;resource_id\
FROM courses\
WHERE user_id IN\
(SELECT\
    &emsp;user_id\
FROM courses\
WHERE resource_type = 'Course' \
AND state = 'successful'\
GROUP BY user_id\
HAVING COUNT(DISTINCT resource_id) > 1)\
AND resource_type = 'Course'\
AND state = 'successful'\
ORDER BY 1

## 2) Обработка данных с помощью Python

In [22]:
# Считаем файл и ознакомимся с содержимым
import pandas as pd
df = pd.read_csv('users_courses.csv')
df.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


Сформируем таблицу со списком всех купленных курсов по каждому пользователю:

In [24]:
users = df.groupby('user_id')['resource_id'].agg(list).reset_index()
users.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [25]:
# Отсортируем номера курсов в списках по возрастанию:
for lst in users.resource_id:
    lst.sort()
users.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [26]:
# Для формирования всех возможные пар из курсов для каждого пользователя 
# импортируем функцию combinations и напишем функцию на её основе

from itertools import combinations

def get_pairs(row):
    return list(combinations(row, 2))      
        

Формируем столбец со списком всех возможных пар курсов по каждому пользователю:

In [27]:
users['pairs'] = users['resource_id'].apply(get_pairs)
users.head()

,user_id,resource_id,pairs
0,51,"[516, 1099]","[(516, 1099)]"
1,6117,"[356, 357, 1125]","[(356, 357), (356, 1125), (357, 1125)]"
2,10275,"[553, 1147]","[(553, 1147)]"
3,10457,"[361, 1138]","[(361, 1138)]"
4,17166,"[356, 357]","[(356, 357)]"


In [28]:
# Чтобы разделить пары курсов, воспользуемся функцией explode()
pairs = users.explode('pairs')
pairs.head()

,user_id,resource_id,pairs
0,51,"[516, 1099]","(516, 1099)"
1,6117,"[356, 357, 1125]","(356, 357)"
1,6117,"[356, 357, 1125]","(356, 1125)"
1,6117,"[356, 357, 1125]","(357, 1125)"
2,10275,"[553, 1147]","(553, 1147)"


Создадим список уникальных пар курсов:

In [29]:
pairs_unique_lst = []
for pair in pairs['pairs']:
    if pair not in pairs_unique_lst:
        pairs_unique_lst.append(pair)
len(pairs_unique_lst)

3989

Всего получилось **3989** уникальных пар

Для подсчёта частотности каждой пары импортируем класс Counter и выведем 30 самых популярных пар:

In [30]:
from collections import Counter
all_pairs_lst = []
for pair in pairs['pairs']:
    all_pairs_lst.append(pair)
pairs_counter = Counter(all_pairs_lst)
pairs_counter.most_common(30)

[((551, 566), 797),
 ((515, 551), 417),
 ((489, 551), 311),
 ((523, 551), 304),
 ((566, 794), 290),
 ((489, 515), 286),
 ((490, 566), 253),
 ((490, 551), 247),
 ((570, 752), 247),
 ((569, 572), 216),
 ((515, 523), 213),
 ((553, 745), 212),
 ((489, 523), 206),
 ((569, 840), 204),
 ((514, 551), 200),
 ((516, 745), 199),
 ((515, 566), 195),
 ((489, 566), 188),
 ((504, 572), 184),
 ((572, 840), 178),
 ((551, 552), 177),
 ((507, 570), 172),
 ((490, 809), 163),
 ((489, 490), 152),
 ((507, 752), 150),
 ((523, 552), 144),
 ((490, 515), 143),
 ((551, 570), 142),
 ((504, 569), 139),
 ((514, 515), 139)]

По этим данным мы можем провести условную границу, что самые популярные пары курсов встречались более 100 раз.

Преобразуем полученные данные по всем парам курсов в датафрейм:

In [31]:
pairs_cnt_df = pd.DataFrame.from_dict(pairs_counter, orient='index').reset_index()
pairs_cnt_df.columns = ['pair', 'cnt']
pairs_cnt_df = pairs_cnt_df.sort_values(by='cnt', ascending=False)
pairs_cnt_df.reset_index(drop=True, inplace=True)
pairs_cnt_df.head()


,pair,cnt
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290


### Для формирования рекомендаций создадим датафрейм с самыми популярными парами. 
Для нас это будут пары курсов, купленные клиентами более 100 раз. \
Всего таких пар 50. 
Так мы исключим из рекомендательной системы курсы, \
не нашедшие достаточной популярности у пользователей.

In [32]:
popular_pairs = pairs_cnt_df[pairs_cnt_df['cnt'] > 100]
len(popular_pairs)

50

In [33]:
# Сформируем список наиболее популярных пар курсов
popular_pairs_lst = list(popular_pairs['pair'])

### Для создания таблицы рекомендаций нам понадобится список всех курсов,
### на основе которого создадим новый датафрейм:

In [34]:
courses_lst = list(df.resource_id.unique())
courses_lst.sort()
recom_table = pd.DataFrame(courses_lst, columns=['course'])
recom_table.head()

,course
0,356
1,357
2,358
3,359
4,360


Создадим список самых популярных курсов на основе созданного ранее списка популярных пар:

In [35]:
popular_courses = []
for lst in popular_pairs_lst:
    for el in lst:
        if el not in popular_courses:
            popular_courses.append(el)
len(popular_courses)

30

### Всего получилось 30 курсов. Именно эти курсы встречались в парах более 100 раз.

## 3) Создание таблицы рекомендаций

Создадим функцию, которая будет рекомендовать по 2 курса для каждого курса из списка.\
Прежде всего функция проверяет, оказался ли курс в одной из самых популярных пар (из списка popular_pairs_lst).\
В рекомендации попадают два самых популярных курса из соответствующих пар.\
Если курс не входит в топ-лист или попал туда только однократно, \
для обеих или только второй рекомендации выбирается один из курсов из списка самых популярных курсов popular_courses.\
Первой рекомендацией будет курс из топ-10, второй - любой из всего списка.\
Таким образом, с одной стороны, мы концентрируем внимание на тех курсах, \
которые больше всего любят пользователи (топ-10), \
а значит, повышаем вероятность совершения пользователем ещё одной покупки. \
С другой стороны, второй рекомендацией из топ-30 мы даём возможность стать известнее курсам, \
которые, похоже, ещё только набирают популярность у пользователей \
и могут стать со временем новыми топовыми рекомендациями.

In [36]:
import random

def get_recommendation(course):
    rec_lst = []
    recs = []
    rec_1 = None
    rec_2 = None
    for pair in popular_pairs_lst:
        if course in pair:
            rec_lst.append([*(pair)])
        if len(rec_lst) == 2:
            break   
    for lst in rec_lst:
        lst.remove(course)
        recs.append(*lst)
    if len(recs) == 1:
        rec_1 = recs[0]
        rec_2 = random.choice(popular_courses[:11])
    elif len(rec_lst) == 0:
        rec_1 = random.choice(popular_courses[:11])
        rec_2 = random.choice(popular_courses)
    else:
        rec_1, rec_2 = recs   
    return [rec_1, rec_2]
        

Используя функцию, создадим столбец с рекомендуемыми курсами и преобразуем его затем в два столбца.

In [37]:

recom_table['recommended'] = list(recom_table['course'].apply(get_recommendation))
recom_table[['recommendation_1','recommendation_2']] = \
    pd.DataFrame(recom_table.recommended.to_list(), index=recom_table.index)
recom_table.drop('recommended', axis=1, inplace=True)
recom_table = recom_table.set_index('course')


Выведем на экран получившуюся таблицу:

In [38]:
recom_table.head(10)

,recommendation_1,recommendation_2
course,,
356,571,566
357,571,515
358,490,840
359,794,357
360,794,553
361,489,356
362,566,570
363,551,553
364,490,679
